In [ ]:
# 기본
import pandas as pd
import numpy as np

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

## 기본 root path 설정 (local 인지 colab인지 확인)

In [ ]:
### local
root_path = '../data/open'

### colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

## 월별 데이터 기본전처리

In [ ]:
### 기본 폴더 구조 설정
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    #"회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    #"신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    #"청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    #"잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    #"채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    #"마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    #"성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

months = ["07","08","09","10","11","12"]

In [ ]:
for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            if split == "test":
                if month != "12":
                    continue
                    
            ### 파일 불러오기
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{root_path}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            df = pd.read_parquet(file_path)

            ### Label Encoding
            # 범주형 -> 수치형
            if category == "회원정보":
                # '_1순위신용체크구분', '_2순위신용체크구분' → 결측치 '기타'로 채우기 및 인코딩
                df['_1순위신용체크구분'] = df['_1순위신용체크구분'].fillna('기타')
                df['_2순위신용체크구분'] = df['_2순위신용체크구분'].fillna('기타')
                mapping = {'신용': 1, '체크': 0, '기타': -1}
                df['1순위신용체크구분_인코딩'] = df['_1순위신용체크구분'].map(mapping)
                df['2순위신용체크구분_인코딩'] = df['_2순위신용체크구분'].map(mapping)
                df.drop(columns=['_1순위신용체크구분','_2순위신용체크구분'], inplace=True)

                # 가입통신회사코드 → 가입통신회사_S사여부
                df['가입통신회사_S사여부'] = (df['가입통신회사코드'] == 'S사').astype(int)
                df.drop(columns=['가입통신회사코드'], inplace=True)

                # 직장시도명 → 직장_수도권여부
                df['직장_수도권여부'] = df['직장시도명'].isin(['서울', '경기']).astype(int)
                df.drop(columns=['직장시도명'], inplace=True)

                # 결측치 많은 변수 삭제
                df.drop(columns=['최종카드발급일자','최종유효년월_신용_이용가능','최종유효년월_신용_이용'], inplace=True)

                ##### 2. 자료형 변환 및 파생변수 생성

                # 거주시도명 → 거주지_수도권여부
                df['거주지_수도권여부'] = df['거주시도명'].isin(['서울', '경기']).astype(int)
                df.drop(columns=['거주시도명'], inplace=True)

                # 연회비발생카드수_B0M → 이진변수화
                df['연회비발생카드수_B0M_이진'] = df['연회비발생카드수_B0M'].isin(['1개이상']).astype(int)
                df.drop(columns=['연회비발생카드수_B0M'], inplace=True)

                # 데이터가 모두 동일한 변수 제거
                columns_drop = ['상품관련면제카드수_B0M','임직원면제카드수_B0M','우수회원면제카드수_B0M','기타면제카드수_B0M']
                df.drop(columns=columns_drop, inplace=True)

                # Life_Stage → 파생변수 생성 후 삭제
                df['Life_Stage_자녀성장_여부'] = df['Life_Stage'].isin(['자녀성장(1)', '자녀성장(2)']).astype(int)
                df.drop(columns=['Life_Stage'], inplace=True)

                # 연령: 숫자만 추출 → int형 변환
                df['연령'] = df['연령'].str.extract(r'(\d+)').astype(float).astype('Int64')

                ##### 3. 상관관계 고려 변수 제거
                columns_drop = ['이용금액_R3M_체크_가족','연회비할인카드수_B0M','할인금액_기본연회비_B0M','할인금액_제휴연회비_B0M']
                df.drop(columns=columns_drop, inplace=True)

                ##### 4. 유사/중복 의미 변수 제거
                columns_drop = ['입회일자_신용','이용카드수_체크_가족','청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M']
                df.drop(columns=columns_drop, inplace=True)
                
            if category == "신용정보":
                # 'RV전환가능여부' → 'RV전환불가능여부'
                df['RV_전환가능여부_이진'] = (df['RV전환가능여부'] == 'N').astype(int)
                df = df.drop(columns='RV전환가능여부')

                ##### 2. 자료형 변환
                # 자발한도감액횟수_R12M: '0회' → 0, '1회' → 1, ...
                df['자발한도감액횟수_R12M'] = (
                    df['자발한도감액횟수_R12M']
                    .str.replace('회', '', regex=False)
                    .astype(int)
                )

                # ‘한도증액횟수_R12M’ → '한도증액_R12M_여부': '0회' → 0, '1회이상' → 1
                df['한도증액_R12M_여부'] = df['한도증액횟수_R12M'].map({
                    '0회': 0,
                    '1회이상': 1
                }).astype(int)
                df.drop(columns=['한도증액횟수_R12M'], inplace=True)

                # '카드론동의여부': 'Y' → 1, 'N' → 0
                df['카드론동의여부'] = df['카드론동의여부'].map({'Y': 1, 'N': 0}).astype(int)

                # '한도심사요청건수' → '한도심사요청여부': '0회' → 0, '1회이상' → 1
                df['한도심사요청여부'] = df['한도심사요청건수'].map({
                    '0회': 0,
                    '1회이상': 1
                }).astype(int)
                df.drop(columns=['한도심사요청건수'], inplace=True)
                
                ##### 4. 변수 별 확인 및 파생 변수 생성
                df['RV실사용여부'] = (df['RV약정청구율'] > 0).astype(int)
                df['강제한도감액횟수_2회이상여부'] = (df['강제한도감액횟수_R12M'] > 1).astype(int)
                df['강제한도감액금액_R12M_3이상여부'] = (df['강제한도감액금액_R12M'] > 2).astype(int)
                df['상향가능CA한도금액_1여부'] = (df['상향가능CA한도금액'] == 1).astype(int)
                
            if category == "승인매출정보":
                df['이용금액대'] = df['이용금액대'].map({
                    '09.미사용' : 0,
                    '05.10만원-' : 1,
                    '04.10만원+' : 2,
                    '03.30만원+' : 3,
                    '02.50만원+' : 4,
                    '01.100만원+' : 5})
                print("🔀 수치형변수로 인코딩")

            if category == "청구정보":
                df['대표청구지고객주소구분코드'] = df['대표청구지고객주소구분코드'].map({
                    '미확인':0, '주거지':1, '회사':2})
                df['대표청구서수령지구분코드'] = df['대표청구서수령지구분코드'].map({
                    '우편':0, '이메일':1, '당사페이앱+이메일':2,'K톡명세서+이메일':3, '미수신':4,
                    '당사멤버십+이메일':5, '문자메세지':6})
                df['청구서수령방법'] = df['청구서수령방법'].map({
                    '우편':0, '이메일':1, '문자메세지':2,
                    'K톡':3, '미수령':4, '당사멤버십':5})
                                                  
            if category == "채널정보":
                df['인입횟수_ARS_R6M'] = df['인입횟수_ARS_R6M'].map({
                    '1회 이상': 0, '10회 이상': 1})
                df['이용메뉴건수_ARS_R6M'] = df['이용메뉴건수_ARS_R6M'].map({
                    '1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
                df['방문횟수_PC_R6M'] = df['방문횟수_PC_R6M'].map({
                    '1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4})
                df['방문일수_PC_R6M'] = df['방문일수_PC_R6M'].map({
                    '1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
                df['방문횟수_앱_R6M'] = df['방문횟수_앱_R6M'].map({
                    '1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4, '50회 이상': 5, '60회 이상': 6, '70회 이상': 7, '80회 이상': 8})

            if category == "마케팅정보":
                df['캠페인접촉건수_R12M'] = df['캠페인접촉건수_R12M'].map({
                    '1회 이상': 0, '5회 이상': 1, '10회 이상': 2,'15회 이상': 3, '20회 이상': 4, '25회 이상': 5})
                df['캠페인접촉일수_R12M'] = df['캠페인접촉일수_R12M'].map({'1일 이상': 0, '5일 이상': 1, '10일 이상': 2, '15일 이상': 3, '20일 이상': 4})
                
            ### 컬럼 제거
            # 모든 값이 동일한 컬럼 제거 (기준년도 컬럼 제외)
            constant_cols = [col for col in df.columns.difference(['기준년월']) if df[col].nunique() == 1]
            if constant_cols:
                print(f"🧹 동일값 컬럼 제거: {constant_cols}")
                df = df.drop(columns=constant_cols)
            
            # 결측치가 있는 컬럼 제거
            null_cols = df.columns[df.isnull().any()]
            if len(null_cols) > 0:
                print(f"🧹 결측치 컬럼 제거: {list(null_cols)}")
                df = df.drop(columns=null_cols)
            
            ### One-hot Encoding
            # segment 값 있는 파일 주소 설정
            segment_folder = f'{root_path}/{split}/1.회원정보'
            segment_category = "회원정보"
            
            # 세그먼트 컬럼 추가
            if split=="train":
                if category!="회원정보":
                    segment_df = pd.read_parquet(f"{segment_folder}/2018{month}_{split}_{segment_category}.parquet")
                    print("🔀 세그먼트 병합")
                    df = df.merge(segment_df[['ID', 'Segment']], on='ID', how='left')
                if category == "신용정보":
                    # 카드이용한도금액_A수준복합 파생변수 생성
                    a_limit = df[df['Segment'] == 'A']['카드이용한도금액_B1M']
                    q1, q3 = a_limit.quantile(0.25), a_limit.quantile(0.75)
                    iqr = q3 - q1
                    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
                    a_B1M = a_limit[(a_limit >= lower) & (a_limit <= upper)].min()

                    a_limit = df[df['Segment'] == 'A']['카드이용한도금액_B2M']
                    q1, q3 = a_limit.quantile(0.25), a_limit.quantile(0.75)
                    iqr = q3 - q1
                    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
                    a_B2M = a_limit[(a_limit >= lower) & (a_limit <= upper)].min()

                    def classify_dual_limit(row):
                        b1 = row['카드이용한도금액_B1M'] >= a_B1M
                        b2 = row['카드이용한도금액_B2M'] >= a_B2M
                        return 2 if b1 and b2 else 1 if b1 or b2 else 0

                    df['카드이용한도금액_A수준복합'] = df.apply(classify_dual_limit, axis=1)
                
                df=pd.get_dummies(df, columns=['Segment'])
            
            # 메모리 최적화
            for col in df.select_dtypes(include='int64').columns:
                if df[col].max() < 2_147_483_647:
                    df[col] = df[col].astype('int32')

            for col in df.select_dtypes(include='float64').columns:
                df[col] = df[col].astype('float32')

            ### 전처리된 파일 저장
            output_file=f"{root_path}/{split}/{folder}/2018{month}_processed_{category}.parquet"
            df.to_parquet(output_file, index=False)
            print(f"✅ 저장 완료: {month}_{split}_{category}")

## 전처리된 월별 데이터 하나로 통합

#### 실행시 메모리 에러 뜨는 경우에는 restart kernel 하신 후 기본폴더구조까지만 실행후 바로 아래 코드 실행해보세요

In [ ]:
split = "train"
merged_list=[]
for category, info in data_categories.items():
    folder = info["folder"]
    suffix = info["suffix"]
    var_prefix = info["var_prefix"]

    for month in months:
        ### 파일 불러오기
        file_path = f"{root_path}/{split}/{folder}/2018{month}_processed_{category}.parquet"
        df = pd.read_parquet(file_path)
        merged_list.append(df)
        print(f"✅ {file_path} 변환 완료")

    # 파일 저장
    if merged_list:
        merged_df = pd.concat(merged_list, ignore_index=True)
        
        # 결측치가 있는 컬럼 제거
        null_cols = merged_df.columns[merged_df.isnull().any()]
        if len(null_cols) > 0:
            print(f"🧹 결측치 컬럼 제거: {list(null_cols)}")
            merged_df = merged_df.drop(columns=null_cols)
        
        output_file = f"{root_path}/{split}/{folder}/cleaned_{suffix}.parquet"
        merged_df.to_parquet(output_file, index=False)
        print(f"✅ 전체 병합 저장 완료: 저장 완료 (Shape: {merged_df.shape})")